In [9]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import KFold, StratifiedKFold
import warnings
import gc
import time
import sys
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

from sklearn.utils import shuffle



In [10]:
train= pd.read_csv("../input/train.csv")
test= pd.read_csv("../input/test.csv")


In [19]:
feats = ["var_{}".format(i) for i in range(200)]
targets2 = ["target{}".format(i) for i in range(0,2)]

split_tar_df = train.copy(deep=True)
split_tar_df['target1'] = train['target']
split_tar_df['target0'] = 1 - train['target']

zero_subset_df = split_tar_df[split_tar_df['target0'] > 0.9]
ones_subset_df = split_tar_df[split_tar_df['target1'] > 0.9]

frames = [zero_subset_df]
for i in range(0,10):
    frames.append(ones_subset_df)

combined_df = pd.concat(frames)
combined_df = shuffle(combined_df)

combined_df.info()

X = combined_df[feats]
X_test = test[feats]
y = combined_df[['target1','target0']]

N_FOLDS = 5.0

kf = KFold(n_splits=int(N_FOLDS), random_state=2019, shuffle=True)

#cvlist = list(StratifiedKFold(5, random_state=12345786).split(X, y))
scaler = StandardScaler()

X_sc = scaler.fit_transform(X)
X_test_sc = scaler.fit_transform(X_test)



<class 'pandas.core.frame.DataFrame'>
Int64Index: 380882 entries, 152983 to 49382
Columns: 204 entries, ID_code to target0
dtypes: float64(200), int64(3), object(1)
memory usage: 595.7+ MB


In [13]:
import keras
from keras.layers import (Flatten, Conv1D, Conv2D, Input, Dense, Dropout, BatchNormalization,
                          concatenate, GaussianNoise, Reshape, TimeDistributed, LeakyReLU, PReLU, Embedding)
from keras.models import Model, load_model, save_model
from keras.optimizers import SGD, Adam
from sklearn.base import BaseEstimator, ClassifierMixin
from pathlib import Path
from keras.callbacks import Callback

class ROC_AUC(Callback):
    def __init__(self, validation_data):
        self.X_val, self.y_val = validation_data
    
    def on_epoch_end(self, epoch, logs={}):
        print("ROC AUC for this fold is ", roc_auc_score(self.y_val, self.model.predict(X_val)))
        
class NNv1(BaseEstimator, ClassifierMixin):
    def __init__(self,
                 inp_shape=200,
                 gaussian_noise=0.01,
                 dense1_dim=32,
                 dense2_dim=32,
                 dense1_kwargs=None,
                 dense2_kwargs=None,
                 classifier_kwargs=None,
                 optimizer=SGD,
                 opt_kwargs=None,
                 ):
        self.inp_shape = inp_shape
        self.gaussian_noise = gaussian_noise
        self.dense1_dim = dense1_dim
        self.dense2_dim = dense2_dim
        self.dense1_kwargs = dense1_kwargs
        self.dense2_kwargs = dense2_kwargs
        self.classifier_kwargs = classifier_kwargs
        self.optimizer = optimizer
        self.opt_kwargs = opt_kwargs
        self._default_initiaization()

    def _default_initiaization(self):
        if self.dense1_kwargs is None:
            self.dense1_kwargs = {"kernel_initializer": "glorot_uniform"}
        if self.dense2_kwargs is None:
            self.dense2_kwargs = {"kernel_initializer": "he_uniform"}
        if self.classifier_kwargs is None:
            self.classifier_kwargs = {"kernel_initializer": "he_uniform"}
        if self.opt_kwargs is None:
            self.opt_kwargs = {}

    def _build_model(self):
        inp = Input(shape=(self.inp_shape,))
        # x = GaussianNoise(self.gaussian_noise)(inp)
        x = Reshape((self.inp_shape, 1))(inp)
        d1 = Dense(self.dense1_dim, activation='tanh',)(x)
        #d1 = TimeDistributed(Dropout(0.2))(d1)
        d2 = Dense(self.dense1_dim, activation='relu',)(x)
        #d2 = PReLU()(d2)
        #d2 = TimeDistributed(Dropout(0.2))(d2)
        x = concatenate([d1, d2])
        x = Flatten()(x)
        out = Dense(1, activation='sigmoid', **self.classifier_kwargs)(x)

        model = Model(inputs=inp, outputs=out)
        opt = self.optimizer(**self.opt_kwargs)
        model.compile(loss='binary_crossentropy', optimizer=opt)
        return model

    def fit(self, X, y, *args, **kwargs):
        self.model = self._build_model()
        print(self.model.summary())
        self.model.fit(X, y, *args, **kwargs)
        return self

    def predict(self, X, y=None, weight_path=None, **kwargs):
        if self.model:
            if weight_path is not None:
                self.model.load_weights(weight_path)
            y_hat = self.model.predict_proba(X, **kwargs)
        else:
            raise ValueError("Model not fit yet")
        return y_hat

Using TensorFlow backend.


In [15]:
y_test_pred = np.zeros(len(test_df))
    
for idx, (train_index, valid_index) in enumerate(kf.split(combined_df)):
    print("Fold {}".format(idx))
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    X_train, X_valid = X_sc.iloc[train_index,:], X_sc.iloc[valid_index,:]
    model.fit(X_train, y_train, epochs=20, batch_size = 512, validation_data =(X_valid, y_valid))
    pred = model.predict_proba(X_valid)
    print(pred.shape)
    print( "  auc = ", roc_auc_score(y_valid, pred) )
    y_test_pred += model.predict(combined_test_df[features])[:,0]
    print(y_test_pred.shape)

#save base submission
sub_df1 = pd.DataFrame({"ID_code":test_df["ID_code"].values})
sub_df1["target"] = y_test_pred / N_FOLDS
sub_df1.to_csv("submission_full_equal_nn.csv", index=False)
#make combined data set for comparison
combined_test_df['nn_equal_out'] = y_test_pred / N_FOLDS
combined_test_df.to_csv("../input/combined_equal_output.csv")

Fold 0


IndexError: positional indexers are out-of-bounds